# Problem

The multipose results are worse than the single pose results. Why?

# Imports

In [ ]:
import harbor.analysis.cross_docking as cd
from pathlib import Path
from importlib import reload
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sp =  cd.DockingDataModel.deserialize("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/full_cross_dock_v2_combined_results/ALL_1_poses.parquet")

In [ ]:
mp = cd.DockingDataModel.deserialize("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/full_cross_dock_v2_combined_results/ALL_50_poses.parquet")

In [ ]:
mpdf = mp.dataframe

In [ ]:
spdf = sp.dataframe

In [ ]:
ev = cd.Evaluator(**json.loads('{"type_":"Evaluator","name":"Evaluator","pose_selector":{"type_":"PoseSelector","name":"Default","category":"PoseSelection","variable":"Pose_ID","ascending":true,"number_to_return":1},"dataset_split":{"type_":"DateSplit","name":"DateSplit","deterministic":false,"split_level":0,"reference_structure_column":"Reference_Structure","n_reference_structures":10,"date_column":"RefData_Date","randomize_by_n_days":1},"similarity_split":null,"scorer":{"type_":"RMSDScorer","name":"RMSD","category":"Score","variable":"PoseData_RMSD","ascending":true,"number_to_return":1},"evaluator":{"type_":"BinaryEvaluation","name":"BinaryEvaluation","variable":"PoseData_RMSD","cutoff":2.0,"below_cutoff_is_good":true},"n_bootstraps":1000,"dataset_before_similarity":true}'))
ev.n_bootstraps = 3

In [ ]:
sp_results = cd.Results.calculate_results(sp, [ev])

In [ ]:
mp_results = cd.Results.calculate_results(mp, [ev])

In [ ]:
sp_results_df = cd.Results.df_from_results(sp_results)
sp_results_df["Source"] = "Single Pose"
mp_results_df = cd.Results.df_from_results(mp_results)
mp_results_df["Source"] = "Multi Pose"

In [ ]:
combined = pd.concat([sp_results_df, mp_results_df], axis=0)

In [ ]:
combined

In [ ]:
sp1 = ev.run_pose_selector([sp])

In [ ]:
mp1 = ev.run_pose_selector([mp])

In [ ]:
sp1 == mp1

In [ ]:
sp1[0].dataframe.nunique()

In [ ]:
mp1[0].dataframe.nunique()

In [ ]:
def extract_and_combine(sp, mp):
    sp_df = sp.dataframe
    mp_df = mp.dataframe
    sp_df["Source"] = "Single Pose"
    mp_df["Source"] = "Multi Pose"
    combined_df = pd.concat([sp_df, mp_df], axis=0)
    return combined_df

In [ ]:
def plot(df, x="PoseData_RMSD", hue="Source"):
    plot_df = df[
    # (plot_df["Query_Ligand"].isin(spdf["Query_Ligand"].unique()[:100]))&
    # (plot_df["Reference_Structure"].isin(spdf["Reference_Structure"].unique()[:100]))
        (df["TanimotoComboData_Aligned"] == True)&
        (df["ECFPData_radius"] == 2)
    ].reset_index()
    fig = sns.displot(plot_df, x=x, hue=hue, kind="ecdf")
    plt.xlim(0, 10)
    return fig

In [ ]:
plot_df = extract_and_combine(sp1[0], mp1[0])
plot_df = plot_df[
    # (plot_df["Query_Ligand"].isin(spdf["Query_Ligand"].unique()[:100]))&
    # (plot_df["Reference_Structure"].isin(spdf["Reference_Structure"].unique()[:100]))
    (plot_df["TanimotoComboData_Aligned"] == True)&
    (plot_df["ECFPData_radius"] == 2)
].reset_index()

In [ ]:
fig = sns.displot(plot_df, x="PoseData_RMSD", hue="Source", kind="ecdf")
# set x axis limits
plt.xlim(0, 10)

In [ ]:
mp2 = ev.run_dataset_split(mp1)
sp2 = ev.run_dataset_split(sp1)

In [ ]:
df2 = extract_and_combine(sp2[0], mp2[0])

In [ ]:
df2.groupby("Source").nunique()

In [ ]:
plot(df2)

In [ ]:
mp3 = ev.run_scorer(mp2)
sp3 = ev.run_scorer(sp2)

In [ ]:
df3 = extract_and_combine(sp3[0], mp3[0])

In [ ]:
plot(df3)

In [ ]:
df3.groupby("Source").nunique()

In [ ]:
mp4 = ev.calculate_results(mp3)
sp4 = ev.calculate_results(sp3)

In [ ]:
mp4

In [ ]:
sp4

# Ok I don't think it's a flaw with my analysis. I *think* it's just because I've changed the settings

These are the settings for the multipose docking:
asap-docking cross-docking \
    --target SARS-CoV-2-Mpro \
    --use-omega \
    --omega-dense \
    --allow-final-clash \
    --posit-method "${posit_method}" \
    --structure-selector "${selector}" \
    --fragalysis-dir ${input_dir} \
    --ligands "${ligandFile2d}" \
    --cache-dir "${prepped_dir}" \
    --output-dir "${compound_name}_docked" \
    --overwrite \
    --no-save-to-cache \
    --use-only-cache \
    --num-poses "${num_poses}" \

These are the settings for the single pose docking:
asap-docking cross-docking \
    --target SARS-CoV-2-Mpro \
    --use-omega \
    --omega-dense \
    --allow-retries \
    --allow-final-clash \
    --relax-mode clash \
    --posit-method "${posit_method}" \
    --structure-selector "${selector}" \
    --fragalysis-dir ${input_dir} \
    --ligands "${ligandFile2d}" \
    --cache-dir "${prepped_dir}" \
    --output-dir "${compound_name}_docked" \
    --overwrite \
    --no-save-to-cache \
    --use-only-cache \
    --num-poses "${num_poses}" \